In [1]:
from application.inception import *
from application.WEIGHTS import *
from application.inception_resnet_v2 import *

weights = InceptionResNetV2_Weights()


In [2]:
img_input = keras.layers.Input(shape=[299, 299, 3])

In [3]:
# x = stem.conv0(img_input)
# print(x.shape)
# x = stem.conv1(x)
# print(x.shape)
# x = stem.conv2(x)
# print(x.shape)
# x = stem.max_pool3(x)
# print(x.shape)
# x = stem.conv4(x)
# print(x.shape)
# x = stem.conv5(x)
# print(x.shape)
# x = stem.max_pool6(x)
# print(x.shape)

In [4]:
x = Stem(weights.STEM)(img_input)
print(x.shape)
x = Block35(weights.BLOCK35)(x)
print(x.shape)
x = ReductionA(weights.REDUCTION_A)(x)
print(x.shape)
x = Block17(weights.BLOCK17)(x)
print(x.shape)
x = ReductionB(weights.REDUCTION_B)(x)
print(x.shape)
x = Block8(weights.BLOCK8)(x)
print(x.shape)


2022-02-02 15:43:01.628712: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-02 15:43:01.628863: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

(None, 35, 35, 320)
(None, 35, 35, 320)
(None, 17, 17, 1088)
(None, 17, 17, 1088)
(None, 8, 8, 2080)
(None, 8, 8, 2080)


In [5]:
model = InceptionResNetV2()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
stem_1 (Stem)                (None, 35, 35, 320)       441920    
_________________________________________________________________
block35_1 (Block35)          (None, 35, 35, 320)       123408    
_________________________________________________________________
block35_2 (Block35)          (None, 35, 35, 320)       123408    
_________________________________________________________________
block35_3 (Block35)          (None, 35, 35, 320)       123408    
_________________________________________________________________
block35_4 (Block35)          (None, 35, 35, 320)       123408    
_________________________________________________________________
block35_5 (Block35)          (None, 35, 35, 320)       123408

In [6]:
import kfood_dataset
batch_size = 32
paths = kfood_dataset.get_image_paths()

dataset path : /Users/hwijin/Desktop/Code/ml/KFood/kfood True
saving classes, labels...
saving crop information...
ready!
finding image paths...
shuffling...
paths ready!


In [7]:
train_set = kfood_dataset.make_kfood_dataset(paths[30000:], n_parse_threads=tf.data.AUTOTUNE, batch_size=batch_size, shuffle_buffer_size=len(paths[30000:]) // batch_size)
valid_set = kfood_dataset.make_kfood_dataset(paths[20000:30000], n_parse_threads=tf.data.AUTOTUNE, batch_size=batch_size, shuffle_buffer_size=10000 // batch_size)
test_set = kfood_dataset.make_kfood_dataset(paths[:20000], n_parse_threads=tf.data.AUTOTUNE, batch_size=None)

In [8]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True, decay=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [9]:
history = model.fit(train_set, steps_per_epoch=10,
          validation_data=valid_set, validation_steps=10, 
          epochs=1
          )

2022-02-02 15:43:13.991242: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-02 15:43:14.020244: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-02 15:43:14.021264: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 15:43:15.579929: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-02-02 15:43:16.280484: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-02-02 15:43:16.879062: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-02-02 15:43:17.506567: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-02-02 15:43:17.542939: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect s

 5/10 [==============>...............] - ETA: 7s - loss: 5.1593 - accuracy: 0.0125  

2022-02-02 15:43:31.506532: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-02-02 15:43:31.506550: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: cHRM chunk does not match sRGB


10/10 [==============================] - ETA: 0s - loss: 5.1682 - accuracy: 0.0125

2022-02-02 15:43:40.103548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 38s 3s/step - loss: 5.1682 - accuracy: 0.0125 - val_loss: 5.0123 - val_accuracy: 0.0125
